## Assignment 4: Build a Neural Network

 1. Find and download the two datasets (identified above).  These dataset must be stored on your Google drive in a subfolder "mnist", that's in folder "data" in your Google drive folder "My Drive".
2. Import any tools that you will need (at least the initial one) into your notebook. More can be imported as needed.
3. Import the datasets from your Google Drive into respective data structures in your notebook.
4. Explore your datasets.  Look at some the data.  How is it organized? This will have a bearing one what you do later.  Look for anomalies.
5. Implement any modifications to the data to deal with the anomalies, if necessary.
6. You may need to convert the type of data structure containing each dataset depending on how you imported your data.
7. Partition your training dataset into a training dataset and a validation dataset.
8. Partition your training dataset into a label dataset and a feature dataset.  Use caution here.  You may want ot scale the features in your training feature dataset.
9. Repeat step 8 for your validation dataset. You may want to scale the features in your validation feature dataset.
10. Repeat step 8 for your test dataset.You may want to scale the features in your test feature dataset.
11. Generate a display (graphic) of a few (3 or 4) of the training data instances.
12. Define a neural network model appropriate to your neural network training problem.
13. Compile your model with the appropriate training parameters.
14. Train  your model using  your training data and your validation data.
15. Display performance statistics produced by the training process.
16. Comment on what step 15 tells you.
17. Run your trained model using the test dataset. Generat and display performance statistics.
18. the last cell of your Colab notebook must provide a narrative describing what you observed through this machine learning process. How well did the model do?  Did it get better as training continued or did it plateau? Would you recommend it for deployment in the "real world"? If so, explain why.  If not, what would  you do to improve the efficacy of your machine learning project?

In [2]:

import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np
import keras
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten
from keras import backend as k
from sklearn.model_selection import train_test_split
import tensorflow as tf



train_set_raw = pd.read_csv("C:\\Users\\Andrew\\Desktop\\472_CNN\\mnist_train.csv")
test_set_raw = pd.read_csv("C:\\Users\\Andrew\\Desktop\\472_CNN\\mnist_test.csv")

data_set = pd.concat([train_set_raw, test_set_raw])


In [3]:
train_set_raw.info()
test_set_raw.info()

train_set_raw.describe()
# Label will be our target variable. (True Numeric representation of the 28x28 pixel matrix)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 59.9 MB


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,60000.000000,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,4.453933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,2.889270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


## Dataset Splitting
I am going to split the data into groups of training testing groups


In [4]:
# FEATURE SET REPRESNETS THE ENTIRE DATASET, Should refactor this to be more clear


labels = data_set.iloc[:, 0]
features = data_set.iloc[:, 1:]

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=42)

# Convert to numpy arrays
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# Reshape the pixel values
img_rows, img_cols = 28, 28

if k.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    inpx = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    inpx = (img_rows, img_cols, 1)

# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print("Unique values in y_train:", np.unique(y_train))
print("Unique values in y_test:", np.unique(y_test))

Unique values in y_train: [0 1 2 3 4 5 6 7 8 9]
Unique values in y_test: [0 1 2 3 4 5 6 7 8 9]


In [5]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [6]:

inpx = Input(shape=inpx)
layer1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inpx)
layer2 = Conv2D(64, (3, 3), activation='relu')(layer1)
layer3 = MaxPooling2D(pool_size=(3, 3))(layer2)
layer4 = Dropout(0.5)(layer3)
layer5 = Flatten()(layer4)
layer6 = Dense(250, activation='sigmoid')(layer5)
layer7 = Dense(10, activation='softmax')(layer6)


In [8]:


model = Model([inpx], layer7)
model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_test, y_test))

Epoch 1/8
 165/3500 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.4384 - loss: 2.1163

KeyboardInterrupt: 